# Intro to Machine Learning Jupyter Notebook

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:
file_path = "StormEvents_details-ftp_v1.0_d2024_c20250401.csv.gz"

df = pd.read_csv(file_path)

df = df[pd.notna(df['DAMAGE_PROPERTY'])] # keep only rows that are not empty in property damage

display(df.head(20))



,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
1,202411,16,230,202411,18,1421,197838,1223377,OREGON,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A series of cold fronts the weekend of Nov. 16...,The Hog Pass SNOTEL reported an estimated 12 i...,CSV
2,202405,19,1839,202405,19,1902,190905,1184919,OKLAHOMA,40,...,5.0,N,CUSTER CITY,35.7100,-99.0010,35.7370,-98.8910,Significant severe weather occurred across por...,While the large multiple-vortex tornado was ap...,CSV
3,202405,23,2155,202405,23,2155,190907,1180805,OKLAHOMA,40,...,2.0,W,NINNEKAH,34.9501,-97.9523,34.9501,-97.9523,Two primary rounds of severe convection occurr...,Relayed image showing a portable outbuilding b...,CSV
4,202405,24,1405,202405,24,1410,191916,1182348,MISSISSIPPI,28,...,0.0,N,ALGOMA,34.1800,-89.0300,34.1800,-89.0300,An active storm pattern as several mid-level s...,Reports of trees down in Algoma.,CSV
5,202411,1,0,202411,1,1600,197531,1221908,WASHINGTON,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A frontal boundary followed by a large area of...,The Swift Creek SNOTEL at 4400 ft recorded 14 ...,CSV
6,202411,1,0,202411,1,1600,197530,1221909,OREGON,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A frontal boundary followed by a large area of...,Bear Grass SNOTEL at 4720 ft recorded 13 inche...,CSV
7,202405,14,1510,202405,14,1515,191820,1181506,MISSISSIPPI,28,...,1.0,NW,TISHOMINGO CO ARPT,34.5100,-88.2100,34.5100,-88.2100,A large upper level low from central Missouri ...,NaN,CSV
8,202405,14,1352,202405,14,1357,191821,1181499,TENNESSEE,47,...,1.0,SSE,EUREKA,35.2000,-88.2400,35.2000,-88.2400,A large upper level low from central Missouri ...,Sub-severe hail reported near Savannah.,CSV
9,202411,17,1100,202411,18,2100,197535,1221931,OREGON,41,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A cold frontal boundary arriving on Nov 17th f...,Bear Grass SNOTEL at 4720 ft recorded 20 inche...,CSV
10,202411,13,100,202411,13,400,197534,1221924,WASHINGTON,53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A closed upper-level low dropped southward int...,Mesonet 82SW1 at Cape Disappointment measured ...,CSV


# Feature Engineering


# Clustering

# Most association with different features